# Labeled Property Graph Play

The notebook looks at working with the LPG from SysML v2 clients

In [1]:
import pymbe.api as pm
from pymbe.label import get_label
import networkx as NX
import matplotlib.pyplot as plt

from pymbe.interpretation.interp_playbooks import random_generator_playbook
from pymbe.interpretation.results import pprint_interpretation

In [2]:
pm.UI.new(host_url="http://sysml2-sst.intercax.com")

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {()}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {('FeatureMembership', 'FeatureValue', 'ParameterMembership', 'ResultExpressionMembership', 'ReturnParameterMembership')}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


DockPop(children=(UI(children=(SysML2ClientWidget(children=(Text(value='http://sysml2-sst.intercax.com', descr…

In [3]:
ui, *_ = _.children
client, tree, inspector, lpg, interpreter = ui.children
client.project_selector.value = client.project_selector.options["Kerbal (2021-04-17 18:03:01)"]
client._download_elements()

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {('FeatureMembership', 'FeatureValue', 'Import', 'Membership', 'ParameterMembership', 'Redefinition', 'ResultExpressionMembership', 'ReturnParameterMembership', 'Superclassing')}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {('FeatureMembership', 'FeatureTyping', 'FeatureValue', 'Import', 'Membership', 'ParameterMembership', 'Redefinition', 'ResultExpressionMembership', 'ReturnParameterMembership')}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {('FeatureTyping', 'FeatureValue', 'Import', 'Membership', 'ParameterMembership', 'Redefinition', 'ResultExpressionMembership', 'ReturnParameterM

In [4]:
lpg.graph

In [5]:
shorten_pre_bake = {
    'RT-10 "Hammer" Solid Fuel Booster': "RT-10",
    'RT-5 "Flea" Solid Fuel Booster': "RT-5",
    'LV-T45 "Swivel" Liquid Fuel Engine': "LV-T45",
    'FL-T100 Fuel Tank': "FL-T100",
    'FL-T200 Fuel Tank': "FL-T200"
}

In [6]:
random_rez = random_generator_playbook(
    lpg,
    shorten_pre_bake
)

Want to look at all the \$results and other parameters

In [7]:
random_rez['ac79381e-74ee-4364-9d20-424a50f4834d']

[]

In [8]:
rpms = [lpg.edges[edg] for edg in list(lpg.edges.keys()) if edg[2] == 'ReturnParameterMembership']

In [9]:
len(rpms)

37

Bind results from member expressions to the input parameters.

In [11]:
eeg = lpg.get_projection("Expression Evaluation Graph")

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {('FeatureTyping', 'Import', 'Membership', 'Redefinition', 'Superclassing', 'FeatureMembership', 'FeatureValue', 'ParameterMembership', 'ResultExpressionMembership')}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


Repackage LPG edges to get a good ID dictionary.

In [12]:
edge_dict = {}
for edge in lpg.edges.values():
    edge_dict.update({edge['@id']: edge})

Develop implied edges to fill in the information from the model about results and input parameters elsewhere in the model.

In [17]:
for rpm in rpms:
    for result_feeder in eeg.predecessors(rpm['memberElement']['@id']):
        # I think what we really want is Expressions that have at least one input parameter
        if 'Expression' in lpg.nodes[result_feeder]['@type'] and lpg.nodes[result_feeder]['@type'] != 'FeatureReferenceExpression':
            result_to_bind = lpg.nodes[result_feeder]['result']
            print(get_label(lpg.nodes[result_feeder], lpg.nodes))
            expr_members = []
            para_members = []
            expr_results = []
            result_members = []
            ownedMemberships = lpg.nodes[result_feeder]['ownedMembership']
            for om in ownedMemberships:
                if 'Parameter' in edge_dict[om['@id']]['@type'] and edge_dict[om['@id']]['@type']:
                    if 'ReturnParameter' in edge_dict[om['@id']]['@type']:
                        result_members.append(edge_dict[om['@id']]['memberElement']['@id'])
                    else:
                        para_members.append(edge_dict[om['@id']]['memberElement']['@id'])
                elif 'Membership' in edge_dict[om['@id']]['@type'] or 'Result' in edge_dict[om['@id']]['@type']:
                    #print(edge_dict[om['@id']])
                    edge_member = edge_dict[om['@id']]['memberElement']['@id']
                    expr_members.append(edge_dict[om['@id']]['memberElement']['@id'])
                    if 'result' in lpg.nodes[edge_member]:
                        expr_result = lpg.nodes[edge_member]['result']['@id']
                        expr_results.append(expr_result)
            edge_stack = []
            for indx, expr in enumerate(expr_members):
                if indx < len(expr_results) and indx < len(para_members):
                    edge_stack.append(
                        get_label(lpg.nodes[expr], lpg.nodes) + ' --[' +
                        lpg.nodes[expr_results[indx]]['@id'] + ']--> ' +
                        get_label(lpg.nodes[para_members[indx]], lpg.nodes)
                    )
            print(edge_stack)

collect ($collection) => $result
['FRE.engines --[4197ab70-699b-4f69-9694-a5f39c2bb0b0]--> $collection']
FRE.Full Mass (p) => $result
['FRE.Full Mass --[dbed26c7-1a89-4e04-ae50-d81ad4886ad6]--> p: Fuel Tank Section']
sum ($collection) => $result
['collect ($collection) => $result --[57be4b8a-285d-41fc-a059-f8e1c3f68b58]--> $collection']
FRE.Mass (p) => $result
['FRE.Mass --[ac79381e-74ee-4364-9d20-424a50f4834d]--> p: Kerbal Rocket Part']
+ ($x, $y) => $result
['sum ($collection) => $result --[d146a0ab-de0c-4d8c-bd25-9913bde2f2e5]--> $x', 'sum ($collection) => $result --[ac16b279-fa4d-4ff4-8584-17e84ea401a2]--> $y']
collect ($collection) => $result
['FRE.tanks --[74c492cc-1104-4ddd-b54c-5fd8a7cab158]--> $collection']
sum ($collection) => $result
['collect ($collection) => $result --[bd31f91a-43a9-42f9-8e69-78c8dd16a03e]--> $collection']


NOTE FOR ABOVE: The FRE.Mass(p) => result type expressions actually have the p as an input, Mass as a "ResultExpressionMembership" (looks like our dot product), and the result of that going to the result.